In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import torch
from torch import nn
import torch.nn.functional as F
import torchvision as tv 
from torchvision import io

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# io.read_image does not support B&W PNG images
def load_sat_image(file):
    return io.read_image(file).to(dtype=torch.float32)#.to(device, dtype=torch.float32)

def load_groundtruth_image(file):
    return torch.as_tensor(mpimg.imread(file), dtype = torch.float32, device=device)

In [ ]:
# Loaded a set of images
root_dir = "data/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(20, len(files)) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [load_sat_image(image_dir + files[i]) for i in range(n)]

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_groundtruth_image(gt_dir + files[i]) for i in range(n)]

n = 10 # Only use 10 images for training

training_images = list(zip(imgs, gt_imgs))[0:n]

In [ ]:
def crop_image(sat_img, gt_img, size) :
    pad = int(size / 2)
    padded = F.pad(sat_img, (pad, pad, pad, pad))
    res = []
    for i in range(gt_img.size(0)):
        for j in range(gt_img.size(1)):
            res.append((padded[:, i : i + size , j : j + size], gt_img[i, j]))
    return res

training_data = []
for sat_img, gt_img in training_images:
    training_data.extend(crop_image(sat_img, gt_img, 31))

print(len(training_data))

In [ ]:
print(training_data[0])

In [ ]:
plt.imshow(training_data[0][0].permute(1, 2, 0).to(dtype=torch.uint8))

In [ ]:
class Model(nn.Module):
    """
        Model that takes a 3 colors 31 x 31 images and 
        returns a single value which correspond to the 
        probability of the center pixel being a road (1)
        or not (0)
    """
    def __init__(self):
        super(Model, self).__init__()
        
        # Input 3 colors 31 x 31 images
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=10,
            kernel_size=15,
            padding=7 # (kernel_size - 1) / 2
        )
        # 10 channels 31 x 31
        self.pool1 = nn.AvgPool2d(
            kernel_size=5,
            padding=2
        )
        # 10 channels 7 x 7 (14 = (31 + 2 * padding - kernel_size) / kernel_size + 1)
        self.conv2 = nn.Conv2d(
            in_channels=10, 
            out_channels=20,
            kernel_size=4,
        )
        # 20 channels 4 x 4 (4 = 7 - kernel_size + 1)
        
        self.lin1 = nn.Linear(20 * 4 * 4, 100)
        self.lin2 = nn.Linear(100, 10)
        self.lin3 = nn.Linear(10, 1)
        
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))

        x = x.view(-1, 20 * 4 * 4)
        
        x = torch.sigmoid(self.lin1(x))
        x = torch.sigmoid(self.lin2(x))
        x = torch.sigmoid(self.lin3(x))
        
        return x

model = Model().to(device=device)
print(model)
print(sum([np.prod(p.size()) for p in model.parameters()]))

In [ ]:
lr = 0.1
criterion = nn.MSELoss()
optimiser = torch.optim.SGD(model.parameters(), lr=lr)

In [ ]:
torch.stack([training_data[0][0]])

In [ ]:
for i in range(len(training_data) // 10) :
    optimiser.zero_grad()
    input = torch.stack([x for x, _ in training_data[i : i + 10]])
    output = model(input)
    loss = criterion(output, torch.stack([y for _, y in training_data[i : i + 10]]).view(10, 1))
    if i % 1600 == 0 :
        print(f"epoch {i} ({i/1600}%), loss = {loss.item()}")
    loss.backward()
    optimiser.step()

In [ ]:
load_sat_image("data/training/images/satImage_001.png")[0][0,1]

In [ ]:
load_background_image("data/training/groundtruth/satImage_001.png").size()

In [ ]:
# Helper functions

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

In [ ]:
# Loaded a set of images
root_dir = "data/training/"

image_dir = root_dir + "images/"
images = tv.datasets.ImageFolder(root=root_dir, transform=tv.transforms.ToTensor())
files = os.listdir(image_dir)
n = min(20, len(files)) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [io.read_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [io.image.read_file(gt_dir + files[i]) for i in range(n)]
print(files[0])

n = 10 # Only use 10 images for training

In [ ]:
images[100]

In [ ]:
gt_imgs[0][0:10]

In [ ]:
print('Image size = ' + str(imgs[0].shape[0]) + ',' + str(imgs[0].shape[1]))

# Show first image and its groundtruth image
cimg = concatenate_images(imgs[0], gt_imgs[0])
fig1 = plt.figure(figsize=(10, 10))
plt.imshow(cimg, cmap='Greys_r')

In [ ]:
import torchvision as tv

tv.io.read_image()

In [ ]:
# Compute features for each image patch
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0

X = np.asarray([ extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

In [ ]:
# Print feature statistics

print('Computed ' + str(X.shape[0]) + ' features')
print('Feature dimension = ' + str(X.shape[1]))
print('Number of classes = ' + str(np.max(Y)))  #TODO: fix, length(unique(Y)) 

Y0 = [i for i, j in enumerate(Y) if j == 0]
Y1 = [i for i, j in enumerate(Y) if j == 1]
print('Class 0: ' + str(len(Y0)) + ' samples')
print('Class 1: ' + str(len(Y1)) + ' samples')

In [ ]:
# Display a patch that belongs to the foreground class
plt.imshow(gt_patches[Y1[3]], cmap='Greys_r')

In [ ]:
# Plot 2d features using groundtruth to color the datapoints
plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=plt.cm.Paired)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    """
        Model that takes a 3 colors 31 x 31 images and 
        returns a single value which correspond to the 
        probability of the center pixel being a road (1)
        or not (0)
    """
    def __init__(self):
        super(Model, self).__init__()
        
        # Input 3 colors 31 x 31 images
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=10,
            kernel_size=15,
            padding=7 # (kernel_size - 1) / 2
        )
        # 10 channels 31 x 31
        self.pool1 = nn.AvgPool2d(
            kernel_size=5,
            padding=2
        )
        # 10 channels 7 x 7 (14 = (31 + 2 * padding - kernel_size) / kernel_size + 1)
        self.conv2 = nn.Conv2d(
            in_channels=10, 
            out_channels=20,
            kernel_size=4,
        )
        # 20 channels 4 x 4 (4 = 7 - kernel_size + 1)
        
        self.lin1 = nn.Linear(20 * 4 * 4, 100)
        self.lin2 = nn.Linear(100, 10)
        self.lin3 = nn.Linear(10, 1)
        
    def forward(self, x):
        print(x.size())
        x = torch.relu(self.conv1(x))
        print(x.size())
        x = self.pool1(x)
        print(x.size())
        x = torch.relu(self.conv2(x))
        print(x.size())

        x = x.view(-1, 20 * 4 * 4)
        
        x = torch.sigmoid(self.lin1(x))
        x = torch.sigmoid(self.lin2(x))
        x = torch.sigmoid(self.lin3(x))
        
        return x

model = Model().to(device=device)
print(model)
print(sum([np.prod(p.size()) for p in model.parameters()]))

In [ ]:
input = torch.randn((1, 3, 31, 31))
out = model(input)
out

In [ ]:
# Predict on the training set
Z = model(X)

# Get non-zeros in prediction and grountruth arrays
Zn = np.nonzero(Z)[0]
Yn = np.nonzero(Y)[0]

TPR = len(list(set(Yn) & set(Zn))) / float(len(Z))
print('True positive rate = ' + str(TPR))

In [ ]:
# Plot features using predictions to color datapoints
plt.scatter(X[:, 0], X[:, 1], c=Z, edgecolors='k', cmap=plt.cm.Paired)

In [ ]:
# Convert array of labels to an image

def label_to_img(imgwidth, imgheight, w, h, labels):
    im = np.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[j:j+w, i:i+h] = labels[idx]
            idx = idx + 1
    return im

def make_img_overlay(img, predicted_img):
    w = img.shape[0]
    h = img.shape[1]
    color_mask = np.zeros((w, h, 3), dtype=np.uint8)
    color_mask[:,:,0] = predicted_img*255

    img8 = img_float_to_uint8(img)
    background = Image.fromarray(img8, 'RGB').convert("RGBA")
    overlay = Image.fromarray(color_mask, 'RGB').convert("RGBA")
    new_img = Image.blend(background, overlay, 0.2)
    return new_img
    

In [ ]:
# Run prediction on the img_idx-th image
img_idx = 12

Xi = extract_img_features(image_dir + files[img_idx])
Zi = logreg.predict(Xi)
plt.scatter(Xi[:, 0], Xi[:, 1], c=Zi, edgecolors='k', cmap=plt.cm.Paired)

In [ ]:
# Display prediction as an image

w = gt_imgs[img_idx].shape[0]
h = gt_imgs[img_idx].shape[1]
predicted_im = label_to_img(w, h, patch_size, patch_size, Zi)
cimg = concatenate_images(imgs[img_idx], predicted_im)
fig1 = plt.figure(figsize=(10, 10)) # create a figure with the default size 
plt.imshow(cimg, cmap='Greys_r')

new_img = make_img_overlay(imgs[img_idx], predicted_im)

plt.imshow(new_img)
